In [ ]:
from CONSTANTS import max_smile_len, min_smile_len,device,char2idx,idx2char,esm_model_path
import pandas as pd
from fine_tune_utils import fine_tune_on_virus,evaluate_generated_vs_known_inhibitors
from  Generator import Generator
from  Discriminator import Discriminator
from train_utilis import load_checkpoint
import torch
from DTI import DrugTargetGNN
import esm
from test_generator import evaluate_generated_vs_known_inhibitors


In [ ]:
sars_cov2_inhibitor_smiles = [
    "CC1=CC=C(C=C1)NC(=O)C2CCC(C2)C3=CC=CC=C3",  # Nirmatrelvir (Paxlovid)
    "CCOC(=O)C1=NC=CN=C1NC2=CC=CC=C2",           # Favipiravir (approx.)
   
]
sars_cov2 = "MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV"
influenza_a_inhibitor_smiles = [
    "CC1(C)C(=O)NC(=O)N(C1C)C2CC2",            # Oseltamivir (Tamiflu)
    "C1CN2CCC1C(C2)NC(=O)C(CO)NC(=O)C3=CC=CC=C3",  # Zanamivir
]
influenza_a= "MSLLTEVETPIRNEWGCRCNDSSDPLVVAASIIGILHLILWILDRLFFKCIYRRFKYGLKRGWISIRRSRIN"  
  


In [ ]:

drugs=pd.read_csv("/kaggle/input/virus-drug/virus_drug_interactions.csv")

drugs = drugs[drugs['SMILES'].str.len().between(min_smile_len, max_smile_len-2)]
drugs=drugs['SMILES'].unique()


### fine tuning


In [ ]:
generator= Generator.Generator()
discriminator= Discriminator.Discriminator()
generator, discriminator = load_checkpoint(generator, discriminator, "checkpoint.pth.tar")
fine_tune_on_virus(influenza_a,drugs,generator,discriminator,)


In [ ]:
DTI_model = DrugTargetGNN()
DTI_model.load_state_dict(torch.load(r"/kaggle/input/drug-target/pytorch/default/2/drug_target_model2.pth", map_location=device))
DTI_model.to(device)
DTI_model.eval()
esm_model, esm_alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
esm_model.eval()

In [ ]:
h=evaluate_generated_vs_known_inhibitors(influenza_a_inhibitor_smiles,generator, DTI_model, esm_model, esm_alphabet, influenza_a, char2idx, idx2char, device)